### Section 200.1: Secure Password Hashing

The [PBKDF2 algorithm](https://en.wikipedia.org/wiki/PBKDF2) exposed by hashlib module can be used to perform secure password hashing. While this algorithm cannot prevent brute-force attacks in order to recover the original password from the stored hash, it makes such attacks very expensive.

In [8]:
import hashlib
import os
salt = os.urandom(16)
hh = hashlib.pbkdf2_hmac('sha256', b'password', salt, 100000)

In [9]:
import binascii
hexhash = binascii.hexlify(hh)

In [10]:
hexhash

b'808a4715a2d803bd323f029ef795114350c053306b4df3883ba188a25dba5c31'

### Section 200.2: Calculating a Message Digest

In [11]:
import hashlib
h = hashlib.new('sha256')
h.update(b'Nobody expects the Spanish Inquisition.')
h.digest()

b'.\xdf\xda\xdaVR[\x12\x90\xff\x16\xfb\x17D\xcf\xb4\x82\xdd)\x14\xff\xbc\xb6Iy\x0c\x0eX\x9eF-='

In [12]:
binascii.hexlify(h.digest())

b'2edfdada56525b1290ff16fb1744cfb482dd2914ffbcb649790c0e589e462d3d'

In [13]:
h.hexdigest()

'2edfdada56525b1290ff16fb1744cfb482dd2914ffbcb649790c0e589e462d3d'

### Section 200.3: Available Hashing Algorithms

In [14]:
import hashlib
hashlib.algorithms_available

{'DSA',
 'DSA-SHA',
 'MD4',
 'MD5',
 'RIPEMD160',
 'SHA',
 'SHA1',
 'SHA224',
 'SHA256',
 'SHA384',
 'SHA512',
 'blake2b',
 'blake2s',
 'dsaEncryption',
 'dsaWithSHA',
 'ecdsa-with-SHA1',
 'md4',
 'md5',
 'ripemd160',
 'sha',
 'sha1',
 'sha224',
 'sha256',
 'sha384',
 'sha3_224',
 'sha3_256',
 'sha3_384',
 'sha3_512',
 'sha512',
 'shake_128',
 'shake_256',
 'whirlpool'}

In [15]:
hashlib.algorithms_guaranteed

{'blake2b',
 'blake2s',
 'md5',
 'sha1',
 'sha224',
 'sha256',
 'sha384',
 'sha3_224',
 'sha3_256',
 'sha3_384',
 'sha3_512',
 'sha512',
 'shake_128',
 'shake_256'}

### Section 200.4: File Hashing

In [18]:
import hashlib
hasher = hashlib.new('sha256')
with open('foo.txt', 'r') as f:
    contents = f.read()
    hasher.update(contents.encode())
hasher.hexdigest()

'2c26b46b68ffc68ff99b453c1d30413413422d706483bfa0f98a5e886266e7ae'

In [21]:
import hashlib
SIZE = 65536
hasher = hashlib.new('sha256')
with open('foo.txt', 'r') as f:
    buffer = f.read(SIZE)
    while len(buffer) > 0:
        hasher.update(buffer.encode())
        buffer = f.read(SIZE)
print(hasher.hexdigest())

2c26b46b68ffc68ff99b453c1d30413413422d706483bfa0f98a5e886266e7ae


### Section 200.5: Generating RSA signatures using pycrypto

In [1]:
import errno
from Crypto.Hash import SHA256
from Crypto.PublicKey import RSA
from Crypto.Signature import PKCS1_v1_5

message = b'This message is from me, I promise.'
try:
    with open('privkey.pem', 'r') as f:
        key = RSA.importKey(f.read())
except IOError as e:
    if e.errno != errno.ENOENT:
        raise
    # No private key, generate a new one. This can take a few seconds.
    key = RSA.generate(4096)
    with open('privkey.pem', 'wb') as f:
        f.write(key.exportKey('PEM'))
    with open('pubkey.pem', 'wb') as f:
        f.write(key.publickey().exportKey('PEM'))
hasher = SHA256.new(message)
signer = PKCS1_v1_5.new(key)
signature = signer.sign(hasher)

In [3]:
with open('pubkey.pem', 'rb') as f:
    key = RSA.importKey(f.read())
hasher = SHA256.new(message)
verifier = PKCS1_v1_5.new(key)
if verifier.verify(hasher, signature):
    print('Nice, the signature is valid!')
else:
    print('No, the message was signed with the wrong private key or modified')

Nice, the signature is valid!


In [6]:
hasher.hexdigest()

'a044e49b435a0acadde9a78e0f80e595c8759dc22c56f6e971b27033a2dc3cb9'

In [10]:
import binascii
binascii.hexlify(signature)

b'9317777e2564280331c8215395fdd321b1f8f0caf39747aca19d02d045d294e7b9a6d61e8c71cde9ee264f9ac8adad9e3a12ef5cd84334f32ad128a297154c884176449ae929f7f45bfdddc128d2a65c6e745589c2c308382950a8f544c684629de9e9629c32d641452b435d34939a1a7e4cac869a07796eeb21408cb0d3a1862669e5f7a2b72b9780f1ddb39c374f7ab619a1826027292f46ae95b365540d8c28d870520d7a6af9383e82cf2df925057e64245644037d4975db6f56f8226b0346c66582b7213fd498f879bd7eeac04a2a104f9073eafe21fafb5acdfbe4a3a7508920f45848d354ceba085f2503f16794df499a192db5b9a49b2542372c5c09e9ede2148428a2a74b8b85427d5dada98cc993f3716eb284d28dcfd81bbd4d102e00df90439962ecc8b4dc54c93a0f9c920d21886011daa2b5032158dc9eb34a1f7ccba71f181b036d532c7265261dcc73479ad9b719c3b8559d66e1e1ad1625be537dfeacc96c68d2cc0c9bb71fb2bd088cf609db52b28fe9123eaa84b0490f5ccac6071ecfe13dc222b9dc6f09403462a8f5007813575832a0c2d5521f569e67c54aed2d4b97e26c72ff1008266cb32ba4e6a061ccea169b5c70205c05e254ebd6ce4023b227427057125e7163d8a8e419dea4130193ecd1050d7bd17157aa259fbb66e0ea322c9e5e4c8701e70c8f4172a3

### Section 200.6: Asymmetric RSA encryption using pycrypto

In [11]:
from Crypto.Cipher import PKCS1_OAEP
from Crypto.PublicKey import RSA
message = b'This is a very secret message.'
with open('pubkey.pem', 'rb') as f:
    key = RSA.importKey(f.read())
cipher = PKCS1_OAEP.new(key)
encrypted = cipher.encrypt(message)

In [12]:
with open('privkey.pem', 'rb') as f:
    key = RSA.importKey(f.read())
cipher = PKCS1_OAEP.new(key)
decrypted = cipher.decrypt(encrypted)

Note: The above examples use PKCS#1 OAEP encryption scheme. pycrypto also implements PKCS#1 v1.5 encryption
scheme, this one is not recommended for new protocols however due to known [caveats.](http://security.stackexchange.com/questions/32050/what-specific-padding-weakness-does-oaep-address-in-rsa)

In [13]:
decrypted

b'This is a very secret message.'

In [15]:
encrypted.hex()

'923ead1b4d58e53346621cd06b446b4ff1612f95c252d9a0c8884f99be4964e133aafd0718bce04ec578e5a44d1848cc28b6df8a152f4f7a9be61bbb64c9f14440c27d958a62fc3b2bf7c4f88a87f25921cb01711eafb6ebbebbd2160018ab0001187a9c612204f156323f5cf67298b9c055d056a470f3891790d90985bd01b7c57292208c8636ca98ca7f55f7085626b4d1366d2bda6bd158ace5b8c015ee7d1863d409c2b13901609b94c2c98242f95da81ccdc8cb2741f6cf299cc010879b419df2aae70b054998dd3b903245e9a0460d5dd78563358e39cd2a97ab238861cc6ca70bb322c9031b45776811d0ca15433922703d3e75bddacaf63b54c485df2b982c99293d89a24bda0625389923d976ea7930445cc0615b7394d82b9639ae2c66ef7783332e6a6fb3883ca42f54c6aa3e9c448562f6f51f7a96dc2dc505129ca43bfb96c76256f880a7304c3fe6e6232331dbff9698ee4dfdbde6952aeb9283be4c01a7dc3bad4d083487f982788f5a7e5a1db876cc4cf6a3d38f194820439968393fbd4c8b4fd3dd86adc1169268f542157336d2bea18f89ff01e05828967d67ac63fa247c783d33eb028c4c09e044fe0280735e83a856afb49053c8e1bbbaedd180bb1201ee179af6bcb9fd22b092b9e82830b33545139f380da06d902a972ed24f4c4bcccacdf42d71c7d88b04b94e5f9

### Section 200.7: Symmetric encryption using pycrypto

In [16]:
import hashlib
import math
import os
from Crypto.Cipher import AES
IV_SIZE = 16 # 128 bit, fixed for the AES algorithm
KEY_SIZE = 32 # 256 bit meaning AES-256, can also be 128 or 192 bits
SALT_SIZE = 16 # This size is arbitrary
cleartext = b'Lorem ipsum'
password = b'highly secure encryption password'
salt = os.urandom(SALT_SIZE)
derived = hashlib.pbkdf2_hmac('sha256', password, salt, 100000, dklen=IV_SIZE + KEY_SIZE)
iv = derived[0:IV_SIZE]
key = derived[IV_SIZE:]
encrypted = salt + AES.new(key, AES.MODE_CFB, iv).encrypt(cleartext)

In [18]:
encrypted.hex()

'ebae70ab82ae4b40fce3bbf74abb378ffbff6104379d173200ddfe'

In [19]:
salt = encrypted[0:SALT_SIZE]
derived = hashlib.pbkdf2_hmac('sha256', password, salt, 100000,
dklen=IV_SIZE + KEY_SIZE)
iv = derived[0:IV_SIZE]
key = derived[IV_SIZE:]
cleartext = AES.new(key, AES.MODE_CFB, iv).decrypt(encrypted[SALT_SIZE:])

In [20]:
cleartext

b'Lorem ipsum'